<a href="https://colab.research.google.com/github/dhruv20047228/Youtube-Trending-Video-Predictor/blob/main/youtube_video_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Neccesary libraries import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from random import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

try:
  data = pd.read_csv('/content/d.csv', on_bad_lines = 'skip')
  print("File loaded successfully, skipping bad lines.")
except pd.errors.ParserError as e:
  print(f"Error reading file: {e}")
  print("Attempting to read with 'engine=python' and on_bad_lines='skip...")
  try:
    data = pd.read_csv('/content/d.csv', engine='python', on_bad_lines='skip')
    print("File loaded successfully using Python engine.")
  except pd.errors.ParserError as e:
    print(f"Error reading file with Python engine: {e}")
    print("Further investigation of the CSV file is required.")

#DecisionTree Of Our Model
class Node:
  def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
    self.feature = feature
    self.threshold = threshold
    self.left = left
    self.right = right
    self.value = value

  def is_leaf_node(self):
    return self.value is not None

class DecisionTree:
  def __init__(self, min_sample_split=4, max_depth=200, n_features=None):
    self.min_sample_split = min_sample_split
    self.max_depth = max_depth
    self.n_features = n_features
    self.root = None

  def fit(self, X, y):
    self.n_features = X.shape[1] if not self.n_features else min(X.shape[1],self.n_features)
    self.root = self._grow_tree(X, y)

  def _grow_tree(self, X, y, depth=0):
    n_samples, n_feats = X.shape
    n_labels = len(np.unique(y))

    if (depth>=self.max_depth or n_labels==1 or n_samples<self.min_sample_split):
      leaf_value = self._most_common_label(y)
      return Node(value=leaf_value)

    feat_idxs = np.random.choice(n_feats, self.n_features, replace=False)

    #Finding best split
    best_feature, best_thresh = self._best_split(X, y, feat_idxs)

    #Creation of Child Nodes
    left_idxs, right_idxs = self._split(X[:, best_feature], best_thresh)
    left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth+1)
    right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth+1)
    return Node(best_feature, best_thresh, left, right)

  def _best_split(self, X, y, feat_idxs):
    best_gain = -1
    split_idx, split_threshold = None, None

    for feat_idx in feat_idxs:
      X_column = X[:, feat_idx]
      thresholds = np.unique(X_column)

      for thr in thresholds:
        #Calculation of information gain
        gain = self._information_gain(y, X_column, thr)

        if gain > best_gain:
          best_gain = gain
          split_idx = feat_idx
          split_threshold = thr

      return split_idx, split_threshold

  def _information_gain(self, y, X_column, threshold):
    #Parent node's Entropy

    parent_entropy = self._entropy(y)

    #Information of Child Nodes
    left_idxs,right_idxs = self._split(X_column, threshold)

    if len(left_idxs) == 0 or len(right_idxs) == 0:
      return 0

    n = len(y)
    n_l, n_r = len(left_idxs), len(right_idxs)
    e_l, e_r = self._entropy(y[left_idxs]), self._entropy(y[right_idxs])
    child_entropy = (n_l/n) * e_l + (n_r/n) * e_r

    #Calculate the Information Gain
    information_gain = parent_entropy - child_entropy
    return information_gain

  def _split(self, X_column, split_thresh):
    left_idxs = np.argwhere(X_column <= split_thresh).flatten()
    right_idxs = np.argwhere(X_column > split_thresh).flatten()
    return left_idxs, right_idxs

  def _entropy(self, y):
    hist = np.bincount(y)
    ps = hist / len(y)
    return -np.sum(p * np.log(p) for p in ps if p>0)

  def _most_common_label(self, y):
    counter = Counter(y)
    if counter:
      value = counter.most_common(1)[0][0]
    else:
      value = None
    return value

  def predict(self, X):
    return np.array([self._traverse_tree(a, self.root) for a in X])

  def _traverse_tree(self, x, node):
    if node.is_leaf_node():
      return node.value

    if x[node.feature] <= node.threshold:
      return self._traverse_tree(x, node.left)
    return self._traverse_tree(x, node.right)


#Random Forest Code
class RandomForest:
  def __init__(self, n_trees = 200, max_depth=10, min_sample_split=3, n_features = None, n_samples=None):
    self.n_trees = n_trees
    self.max_depth = max_depth
    self.min_sample_split = min_sample_split
    self.n_features = n_features
    self.trees = []
    self.n_samples = n_samples

  def fit(self, X, y):
    self.trees = []

    for _ in range(self.n_trees):
      tree = DecisionTree(max_depth=self.max_depth, min_sample_split=self.min_sample_split, n_features=self.n_features)
      X_sample, y_sample = self._bootstrap_samples(X, y)
      tree.fit(X_sample, y_sample)
      self.trees.append(tree)

  def _bootstrap_samples(self, X, y):
    #Tends to be change
    n_samples = X.shape[0]

    if self.n_samples is None:
      self.n_samples = n_samples
    idxs = np.random.choice(n_samples, self.n_samples, replace=True)
    return X[idxs], y[idxs]

  def _most_common_label(self, y):
    counter = Counter(y)
    most_common = counter.most_common(8)[0][0]
    return most_common

  def predict(self, X):
    predictions = np.array([tree.predict(X) for  tree in self.trees])
    tree_preds = np.swapaxes(predictions, 0, 1)
    predictions = np.array([self._most_common_label(pred) for pred in tree_preds])
    return predictions


#Train
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

#LabelEnCoder Added
label_encoder = LabelEncoder()

y = label_encoder.fit_transform(y)


X_train, X_test , y_train, y_test = train_test_split(
    X, y, test_size = 0.2, random_state = 1234
)
def accuracy(y_true, y_pred):
  accuracy = np.sum(y_true==y_pred)/len(y_true)
  return accuracy

clf = RandomForest(n_trees=200)
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
predictions = label_encoder.inverse_transform(predictions)
print(predictions)
acc = accuracy(y_test, predictions)
print("The accuracy is : ", acc)

#Train
X = data.data
y = data.target

X_train, X_test , y_train, y_test = train_test_split(
    X, y, test_size = 0.2, random_state = 1234
)
def accuracy(y_true, y_pred):
  accuracy = np.sum(y_true==y_pred)/len(y_true)
  return accuracy

clf = RandomForest(n_trees=200)
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
print(predictions)
acc = accuracy(y_test, predictions)
print("The accuracy is : ", acc)